In [1]:
%pip install vertica-python
%pip install clickhouse-driver



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Clickhouse

Таблица создана через clickhouse-client в контейнере:
CREATE TABLE player_events ( `event_time` DateTime, `user_id` UInt64, `movie_id` UInt64, `event_type` Enum8('resolution_change' = 1, 'like' = 2, 'dislike' = 3, 'page_close' = 4, 'watch_complete' = 5, 'comment' = 6),`resolution` String DEFAULT '',`comment_text` String DEFAULT '') ENGINE = ReplicatedMergeTree('/clickhouse/tables/shard1/test', 'replica_1') PARTITION BY toYYYYMMDD(event_time) ORDER BY (event_time, user_id, movie_id)

In [2]:
import time
import random
from clickhouse_driver import Client

client = Client('localhost')

In [4]:
total_records = 10_000_000
batch_size = 10_000

event_types = ['resolution_change', 'like', 'dislike', 'page_close', 'watch_complete', 'comment']

def generate_data(batch_size):
    data = []
    current_time = int(time.time())
    for i in range(batch_size):
        data.append((
            i + 1,
            random.randint(1, 1000000),  # user_id
            random.randint(1, 10000),  # movie_id
            random.choice(event_types),  # event_type
            current_time  # event_time
        ))
    return data

start_time = time.time()
total_batches = total_records // batch_size

for batch_number in range(total_batches):
    data_batch = generate_data(batch_size)
    
    batch_start_time = time.time()
    
    client.execute('INSERT INTO player_events (id, user_id, movie_id, event_type, event_time) VALUES', data_batch)
    
    batch_end_time = time.time()
    
    batch_time = batch_end_time - batch_start_time
    
    progress = (batch_number + 1) / total_batches * 100
    avg_speed = (batch_number + 1) / (time.time() - start_time)
    
    print(f'Пачка {batch_number + 1}/{total_batches} загружена за {batch_time:.2f} сек. Прогресс: {progress:.2f}%. '
          f'Средняя скорость: {avg_speed:.2f} пачек/сек')

total_time = time.time() - start_time
print(f'Загрузка завершена. Общее время: {total_time:.2f} сек.')

Пачка 1/1000 загружена за 5.76 сек. Прогресс: 0.10%. Средняя скорость: 0.17 пачек/сек
Пачка 2/1000 загружена за 0.13 сек. Прогресс: 0.20%. Средняя скорость: 0.33 пачек/сек
Пачка 3/1000 загружена за 0.12 сек. Прогресс: 0.30%. Средняя скорость: 0.48 пачек/сек
Пачка 4/1000 загружена за 0.20 сек. Прогресс: 0.40%. Средняя скорость: 0.62 пачек/сек
Пачка 5/1000 загружена за 0.15 сек. Прогресс: 0.50%. Средняя скорость: 0.74 пачек/сек
Пачка 6/1000 загружена за 0.09 сек. Прогресс: 0.60%. Средняя скорость: 0.87 пачек/сек
Пачка 7/1000 загружена за 0.13 сек. Прогресс: 0.70%. Средняя скорость: 0.99 пачек/сек
Пачка 8/1000 загружена за 0.14 сек. Прогресс: 0.80%. Средняя скорость: 1.10 пачек/сек
Пачка 9/1000 загружена за 0.10 сек. Прогресс: 0.90%. Средняя скорость: 1.21 пачек/сек
Пачка 10/1000 загружена за 0.15 сек. Прогресс: 1.00%. Средняя скорость: 1.30 пачек/сек
Пачка 11/1000 загружена за 0.10 сек. Прогресс: 1.10%. Средняя скорость: 1.40 пачек/сек
Пачка 12/1000 загружена за 0.16 сек. Прогресс: 1.20%

In [6]:
aggregation_query = '''
    SELECT
        user_id,
        event_type,
        COUNT(*) AS event_count
    FROM player_events
    GROUP BY user_id, event_type
    ORDER BY user_id, event_type
'''

num_iterations = 100
total_time = 0

for i in range(num_iterations):
    read_start_time = time.time()
    
    result = client.execute(aggregation_query)
    
    read_end_time = time.time()

    query_time = read_end_time - read_start_time
    total_time += query_time
    
    if (i + 1) % 10 == 0:
        print(f'{i + 1} запросов выполнено.')

average_time = total_time / num_iterations
print(f'Среднее время выполнения запроса с агрегацией за {num_iterations} запусков: {average_time:.4f} сек.')

10 запросов выполнено.
20 запросов выполнено.
30 запросов выполнено.
40 запросов выполнено.
50 запросов выполнено.
60 запросов выполнено.
70 запросов выполнено.
80 запросов выполнено.
90 запросов выполнено.
100 запросов выполнено.
Среднее время выполнения запроса с агрегацией за 100 запусков: 9.3593 сек.


Скорость парралельной загрузки и чтения измеряются в ноутбуках read_data и insert_data

Vertica

In [8]:
import vertica_python

conn_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',  # стандартный пользователь
    'password': '',  # если пароль не был установлен
    'database': 'docker',  # имя базы данных, созданной в контейнере
    'autocommit': True
}

connection = vertica_python.connect(**conn_info)

cursor = connection.cursor()
cursor.execute("SELECT version()")
result = cursor.fetchone()
print("Версия Vertica:", result[0])

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\vertica_python\vertica\connection.py:629: UserWarning: TLS is not configured on the server. Proceeding with an unencrypted channel.
HINT: Set connection option 'tlsmode' to 'disable' to explicitly create a non-TLS connection.
  warnings.warn(msg + hint)


Версия Vertica: Vertica Analytic Database v9.2.0-7


In [9]:
create_table_query = """
CREATE TABLE IF NOT EXISTS player_events (
    user_id INT,
    movie_id INT,
    event_type VARCHAR(50),
    event_time TIMESTAMP
);
"""

cursor.execute(create_table_query)
print("Таблица player_events успешно создана.")

Таблица player_events успешно создана.


In [11]:
from datetime import datetime

def generate_data(batch_size):
    data = []
    event_types = ['resolution_change', 'like', 'dislike', 'page_closed', 'finished', 'comment']
    for _ in range(batch_size):
        user_id = random.randint(1, 1000000)
        movie_id = random.randint(1, 10000)
        event_type = random.choice(event_types)
        event_time = datetime.now()
        data.append((user_id, movie_id, event_type, event_time))
    return data


def insert_data(total_rows, batch_size):
    connection = vertica_python.connect(**conn_info)
    cursor = connection.cursor()

    total_batches = total_rows // batch_size
    total_insert_time = 0

    for batch_num in range(total_batches):
        data_batch = generate_data(batch_size)
        
        query = '''
            INSERT INTO player_events (user_id, movie_id, event_type, event_time)
            VALUES (%s, %s, %s, %s)
        '''
        
        start_time = time.time()
        cursor.executemany(query, data_batch)
        insert_time = time.time() - start_time

        total_insert_time += insert_time
        print(f'Партия {batch_num + 1} вставлена за {insert_time:.4f} сек.')

    cursor.close()
    connection.close()

    average_time_per_batch = total_insert_time / total_batches
    print(f'Среднее время вставки одной партии данных: {average_time_per_batch:.4f} сек.')

total_rows = 10_000_000
batch_size = 1000

insert_data(total_rows, batch_size)

Партия 1 вставлена за 0.1324 сек.
Партия 2 вставлена за 0.0474 сек.
Партия 3 вставлена за 0.0443 сек.
Партия 4 вставлена за 0.0369 сек.
Партия 5 вставлена за 0.0356 сек.
Партия 6 вставлена за 0.0388 сек.
Партия 7 вставлена за 0.0396 сек.
Партия 8 вставлена за 0.0364 сек.
Партия 9 вставлена за 0.0381 сек.
Партия 10 вставлена за 0.0526 сек.
Партия 11 вставлена за 0.0456 сек.
Партия 12 вставлена за 0.0392 сек.
Партия 13 вставлена за 0.0338 сек.
Партия 14 вставлена за 0.0348 сек.
Партия 15 вставлена за 0.0402 сек.
Партия 16 вставлена за 0.0267 сек.
Партия 17 вставлена за 0.0412 сек.
Партия 18 вставлена за 0.0397 сек.
Партия 19 вставлена за 0.0405 сек.
Партия 20 вставлена за 0.0348 сек.
Партия 21 вставлена за 0.0292 сек.
Партия 22 вставлена за 0.0553 сек.
Партия 23 вставлена за 0.0764 сек.
Партия 24 вставлена за 0.0515 сек.
Партия 25 вставлена за 0.0990 сек.
Партия 26 вставлена за 0.0639 сек.
Партия 27 вставлена за 0.0666 сек.
Партия 28 вставлена за 0.0531 сек.
Партия 29 вставлена за 0.0542

In [12]:
aggregation_query = '''
    SELECT user_id, event_type, COUNT(*) AS event_count
    FROM player_events
    GROUP BY user_id, event_type
    ORDER BY user_id, event_type
'''

num_iterations = 10
total_time = 0

for i in range(num_iterations):

    start_time = time.time()
    

    cursor.execute(aggregation_query)
    result = cursor.fetchall()
    

    end_time = time.time()
    query_time = end_time - start_time
    total_time += query_time
    
    print(f'Запрос {i + 1} выполнен за {query_time:.4f} сек.')


average_time = total_time / num_iterations
print(f'Среднее время выполнения запроса: {average_time:.4f} сек.')


cursor.close()
connection.close()

Запрос 1 выполнен за 178.5385 сек.
Запрос 2 выполнен за 172.8596 сек.
Запрос 3 выполнен за 150.2733 сек.
Запрос 4 выполнен за 117.3903 сек.
Запрос 5 выполнен за 137.5781 сек.
Запрос 6 выполнен за 146.9028 сек.
Запрос 7 выполнен за 140.4698 сек.
Запрос 8 выполнен за 142.5441 сек.
Запрос 9 выполнен за 180.5923 сек.
Запрос 10 выполнен за 148.6531 сек.
Среднее время выполнения запроса: 151.5802 сек.


Скорость парралельной загрузки и чтения измеряются в ноутбуках read_data и insert_data

Сводная таблица:

| Параметры | Clickhouse | Vertica |
|-------------|-------------|-------------|
| Средняя скорость загрузки данных (по 1000) | 0.217  | 0.0561  |
| Средняя скорость запроса агрегации  | 9.3593 | 151.5802 |
| Средняя скорость загрузки данных под нагрузкой  | 0.2384  | 0.1064  |
| Средняя скорость чтения данных под нагрузкой  | 12.3215  | 220.7443  |
